# 11 · Figures & Tables

Generate publication-ready plots and LaTeX tables summarizing membership inference leakage.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)

In [ ]:
import pandas as pd

REPORTS_DIR = PROJECT_ROOT / 'reports'
FIGS_DIR = REPORTS_DIR / 'figs'
TABLES_DIR = REPORTS_DIR / 'tables'
FIGS_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

metrics_path = REPORTS_DIR / f'metrics_core_{RUN_MODE.name}.csv'
if not metrics_path.exists():
    raise FileNotFoundError('Consolidated metrics missing. Run notebook 06 before generating figures.')

metrics_df = pd.read_csv(metrics_path)
if metrics_df.empty:
    raise ValueError('Metrics dataframe is empty.')

metrics_df = metrics_df.sort_values(['track', 'slice_id'])
summary_table_path = TABLES_DIR / f'core_attacks_{RUN_MODE.name}.csv'
metrics_df.to_csv(summary_table_path, index=False)
print(f'Saved summary table to {summary_table_path}')

results_csv = REPORTS_DIR / 'results.csv'
metrics_long = metrics_df.melt(
    id_vars=['slice_id', 'track', 'run_mode'],
    value_vars=['auc', 'tpr_at_0.01'],
    var_name='metric',
    value_name='value',
)
metrics_long.to_csv(results_csv, index=False)
print(f'Updated consolidated results at {results_csv}')


In [ ]:
import matplotlib.pyplot as plt

tracks = metrics_df['track'].unique().tolist()

fig, ax = plt.subplots(figsize=(6, 4))
for track in tracks:
    subset = metrics_df[metrics_df['track'] == track]
    ax.plot(subset['slice_id'], subset['auc'], marker='o', label=track)
ax.set_xlabel('Slice ID')
ax.set_ylabel('AUC')
ax.set_title('Core Attack AUC by Slice')
ax.grid(True, linestyle='--', alpha=0.4)
ax.legend()
auc_fig = FIGS_DIR / f'core_auc_{RUN_MODE.name}.png'
fig.savefig(auc_fig, dpi=200, bbox_inches='tight')
plt.close(fig)
print(f'Saved AUC plot to {auc_fig}')

fig, ax = plt.subplots(figsize=(6, 4))
for track in tracks:
    subset = metrics_df[metrics_df['track'] == track]
    ax.plot(subset['slice_id'], subset['tpr_at_0.01'], marker='o', label=track)
ax.set_xlabel('Slice ID')
ax.set_ylabel('TPR @ 1% FPR')
ax.set_title('Core Attack TPR@1%FPR by Slice')
ax.grid(True, linestyle='--', alpha=0.4)
ax.legend()
tpr_fig = FIGS_DIR / f'core_tpr_{RUN_MODE.name}.png'
fig.savefig(tpr_fig, dpi=200, bbox_inches='tight')
plt.close(fig)
print(f'Saved TPR plot to {tpr_fig}')


Exported tables + figures reflecting metrics from notebooks 05-09.
